In [1]:
!pip install fastapi nest_asyncio uvicorn pyngrok diffusers transformers torch accelerate gtts


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [2]:
!ngrok config add-authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:

import os, json, textwrap, base64, requests, nest_asyncio, threading, tempfile
from pathlib import Path
from typing import List
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel, Field
from google import generativeai as genai
from diffusers import StableDiffusionXLPipeline
from gtts import gTTS
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips
from pyngrok import ngrok
import uvicorn, torch
from google import genai
from langchain.prompts import PromptTemplate
import torch
from fastapi import HTTPException
from dotenv import load_dotenv
import traceback
from fastapi.responses import JSONResponse
from fastapi import BackgroundTasks
import uuid
from fastapi.responses import FileResponse
import threading
from gtts import gTTS, gTTSError
import time
import time
import gc
from io import BytesIO
# ----------- USER SETTINGS -----------------------------------------------
# — API keys ——————————————————————————————————————————————————————


load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
client = genai.Client()
REMOTE_URL   = ""   # <- from Colab B
LOCAL_RATIO  = 0.40                                # 40 % images here
GEN_MODEL    = "gemini-2.0-flash"
PORT         = 9000
# -------------------------------------------------------------------------

# ----------- load heavy assets once --------------------------------------
print("⚙️ Loading SDXL‑Turbo…")
import torch
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=DTYPE,          # ← use the real dtype
        variant="fp16"              # keeps weights in half‑precision
).to("cuda" if torch.cuda.is_available() else "cpu")


# ----------- Pydantic schemas --------------------------------------------
class ScenePlan(BaseModel):
    scene_number: int
    image_prompt: str
    story_chunk: str


class Scenes(BaseModel):
    scenes: List[ScenePlan]
    total_number_of_scenes: int
class StoryRequest(BaseModel):
    story      : str = Field(..., description="Full Urdu story")

class StoryResponse(BaseModel):
    video_url : str
# -------------------------------------------------------------------------
ghibli_story_image_prompt_generator = PromptTemplate(
    input_variables=["story_concept"], # Expecting num_scenes around 24 now
    template="""
You are a master Storyboard Artist and Sequence Director, blending the evocative environmental storytelling of Studio Ghibli with a strong cinematic understanding of narrative structure and pacing. Your role is to break down a **Urdu-language story** into a coherent **visual sequence of scenes** that align with the story’s emotional and narrative beats.

# 🎯 Task Overview:
Given an input Urdu **story** and a specified number of scenes (that you find appropiate), perform the following:

---

## 🖼️ OBJECTIVES:

### 1. SCENE VISUALIZATION
Generate number of scenes according to the requirement of story but keep it under 12. For each scene create detailed image prompts**, each corresponding to a distinct **scene number**. These prompts should:
- Translate the narrative moment into a vivid **cinematic visual**.
- Be written as rich, **Studio Ghibli-style descriptions** (suitable for AI generation tools like Stable Diffusion).
- Reflect the tone, mood, and pacing of the original story.

### 2. STORY SEGMENTATION (for TTS)
Segment the original **Urdu story text** {story_concept} into **total number of scenes distinct story chunks**, without altering or rewriting the text. Each chunk should:
- Match the **corresponding scene**.
- Maintain the **original phrasing**, but add **minor TTS-friendly cues** like natural pauses (`۔۔۔`, ellipses, line breaks) to guide expressive reading.
- End at natural narrative breaks such as paragraph ends, dramatic pauses, or shifts in time/location.

---

## 🧠 HOW TO STRUCTURE YOUR THINKING

- Carefully read the full Urdu story and mentally map its plot structure.
- Identify the **main narrative beats**: beginning, conflict, development, climax, and resolution.
- Thoughtfully map all scenes across these beats.
- Major emotional or visual moments may require multiple scenes.

---

## 🎨 IMAGE PROMPT GENERATION GUIDELINES

- Each **image prompt** must describe a **single, vivid cinematic moment** from the story.
- Use a painterly, cinematic style inspired by:
  **Studio Ghibli, art by Hayao Miyazaki, Kazuo Oga background art, anime aesthetic, painterly, atmospheric perspective, intricate detail.**
- Consider:
  - Emotion of the scene
  - Environment and atmosphere
  - Lighting and weather
  - Camera angles and composition
  - Continuity with prior scenes

** Rules that must be followed**
1) The contents or the wording of the story {story_concept} will not be changed.
2) The orignal story {story_concept} will be divided into chunks. Not to be confused by making a new story entirely. It will be the orignal story but in a chunk of smaller sentences.
3) Orignal Narrative {story_concept} will not be changed.
4) The image prompt will be a depiction of the chunk of the story. But they will not be character centric. Rather focus on the environment and the sceneray.

Keep each image prompt around **70–77 tokens**. Write in **natural, vivid paragraphs**.
Also return the total number of scenes that you think the story should haev. the number should be below 12.
---

## 🧾 OUTPUT FORMAT (JSON)

Return a list of scenes dictionaries**
Make sure that the image descriptions that you provide are more scene centric than character because we will not be able to get the same main character everytime. So if we are displaying characters they should be side characters or backgroud characters
"""

)

def plan_scenes(story: str) -> List[ScenePlan]:
    formatted_prompt = ghibli_story_image_prompt_generator.format(story_concept=story)
    res = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=formatted_prompt,
    config={
        'response_mime_type': 'application/json',
        'response_schema': Scenes,
    },
)
    response_json = json.loads(res.text)
    return response_json

def make_image_local(scene: dict, out: Path) -> str:
    try:
        # Define output path
        p = out / f"scene_{scene['scene_number']:02}.png"

        # Generate image
        image = pipe(scene["image_prompt"], num_inference_steps=25, height=1000, width=1600).images[0]

        # Save to disk
        image.save(p)

        # 🧹 Cleanup
        del image
        torch.cuda.empty_cache()
        gc.collect()

        return str(p)
    except Exception as e:
        print(f"❌ Error generating local image for scene {scene['scene_number']}: {e}")
        raise

def request_remote(chunk: List[dict]) -> dict[int, bytes]:
    out = {}
    for s in chunk:
        for attempt in range(3):  # Try up to 3 times
            try:
                r = requests.post(
                    REMOTE_URL + "/generate_images",
                    json=[s],
                    timeout=300
                )
                r.raise_for_status()
                out.update({int(k): base64.b64decode(v) for k, v in r.json().items()})
                break  # ✅ success
            except Exception as e:
                print(f"❌ Scene {s['scene_number']} failed (attempt {attempt+1}): {e}")
                if attempt == 2:  # All attempts failed
                    raise
                time.sleep(3)
    return out


def safe_tts(text, path, retries=3, delay=2):
    for attempt in range(retries):
        try:
            gTTS(text, lang='ur', slow=False).save(path)
            return
        except gTTSError as e:
            print(f"⚠️ gTTS error (attempt {attempt+1}):", e)
            time.sleep(delay)
    raise RuntimeError("gTTS failed after retries")



def keep_colab_b_alive():
    while True:
        try:
            requests.get(REMOTE_URL + "/docs", timeout=5)
        except:
            pass
        time.sleep(30)

threading.Thread(target=keep_colab_b_alive, daemon=True).start()

# -------------------------------------------------------------------------

# ----------- FastAPI -----------------------------------------------------
app = FastAPI(title="Storyboard→Video API")
FILES_DIR = Path("output"); FILES_DIR.mkdir(exist_ok=True)
app.mount("/files", StaticFiles(directory=FILES_DIR), name="files")

def keep_colab_a_alive():
    while True:
        try:
            requests.get(REMOTE_URL + "/docs", timeout=5)
        except:
            pass
        time.sleep(30)

threading.Thread(target=keep_colab_b_alive, daemon=True).start()

job_store = {}  # job_id → dict(status, video_path, error)

@app.post("/make_video")
def make_video(req: StoryRequest, bg: BackgroundTasks):
    job_id = str(uuid.uuid4())
    job_store[job_id] = {"status": "processing"}
    bg.add_task(process_video_job, job_id, req)
    return {"job_id": job_id}
def process_video_job(job_id, req: StoryRequest):
    try:

        response_json = plan_scenes(req.story)
        scenes = response_json["scenes"]
        num_scenes=response_json["total_number_of_scenes"]

        local_cnt = max(1, round(num_scenes * LOCAL_RATIO))
        local_chunk = scenes[:local_cnt]
        remote_chunk = scenes[local_cnt:]

        assets_dir = FILES_DIR / f"assets_{job_id}"
        (assets_dir / "audio").mkdir(parents=True, exist_ok=True)
        img_paths, audio_paths = {}, {}

        for s in local_chunk:
            img_paths[s["scene_number"]] = make_image_local(s, assets_dir)

        if remote_chunk:
            remote_images = request_remote(remote_chunk)
            for sn, img_bytes in remote_images.items():
                p = assets_dir / f"scene_{sn:02}.png"
                p.write_bytes(img_bytes)
                img_paths[sn] = str(p)

        for s in scenes:
            mp3 = assets_dir / "audio" / f"scene_{s['scene_number']:02}.mp3"
            safe_tts(s["story_chunk"], str(mp3))
            audio_paths[s["scene_number"]] = str(mp3)

        clips = []
        for s in sorted(scenes, key=lambda s: s["scene_number"]):
            img = img_paths[s["scene_number"]]
            aud = audio_paths[s["scene_number"]]
            dur = AudioFileClip(aud).duration
            clip = ImageClip(img).set_audio(AudioFileClip(aud))\
                                 .set_duration(dur).resize(height=1080)
            clips.append(clip)

        vid_path = FILES_DIR / f"{job_id}.mp4"
        concatenate_videoclips(clips, method="compose")\
            .write_videofile(str(vid_path), fps=24, codec="libx264", audio_codec="aac")

        job_store[job_id] = {"status": "done", "video_path": str(vid_path)}
    except Exception as e:
        import traceback
        traceback.print_exc()
        job_store[job_id] = {"status": "error", "error": str(e)}
@app.get("/job_status/{job_id}")
def check_status(job_id: str):
    job = job_store.get(job_id)
    if not job:
        raise HTTPException(status_code=404, detail="Invalid job_id")
    return {"status": job["status"]}

@app.get("/get_video/{job_id}")
def get_video(job_id: str):
    job = job_store.get(job_id)
    if not job or job["status"] != "done":
        raise HTTPException(status_code=404, detail="Video not ready")
    return FileResponse(job["video_path"], media_type="video/mp4")

FILES_DIR = Path("output")
FILES_DIR.mkdir(exist_ok=True)
app.mount("/files", StaticFiles(directory=FILES_DIR), name="files")




  if event.key is 'enter':



⚙️ Loading SDXL‑Turbo…


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
import nest_asyncio, threading, time, socket
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()
PORT = 9001

# 1️⃣ Start FastAPI server in a background thread
def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="info")

threading.Thread(target=run_uvicorn, daemon=True).start()

# 2️⃣ Wait for the server to be ready
print("⏳ Waiting for Uvicorn to be ready...")
for _ in range(30):  # 15 sec max
    with socket.socket() as s:
        if s.connect_ex(("127.0.0.1", PORT)) == 0:
            break
    time.sleep(0.5)
else:
    raise RuntimeError("❌ Uvicorn did not start on port 9000!")

# 3️⃣ Create the tunnel only AFTER FastAPI is ready
public_url = ngrok.connect(PORT, bind_tls=False).public_url
print("✅ Public API is live at:", public_url)
print("▶️ Try:", public_url + "/docs")


INFO:     Started server process [441]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9001 (Press CTRL+C to quit)


⏳ Waiting for Uvicorn to be ready...
✅ Public API is live at: http://dc22-34-142-242-33.ngrok-free.app
▶️ Try: http://dc22-34-142-242-33.ngrok-free.app/docs


In [6]:
%%javascript
function keepAlive() {
  setInterval(() => {
    google.colab.kernel.invokeFunction('notebook.ping', [], {});
    console.log("⏳ Keeping Colab alive...");
  }, 60000);  // every 60 seconds
}
keepAlive();

INFO:     34.142.242.33:0 - "POST /make_video HTTP/1.1" 200 OK
🎬 Job submitted. ID: dc0302c1-977c-42cc-8d16-eb6518d12e71
⏳ Waiting for job to finish...
INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK
Status: processing
INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK


  0%|          | 0/25 [00:00<?, ?it/s]

Status: processing
INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK
Status: processing
INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK
Status: processing
INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK
Status: processing


  0%|          | 0/25 [00:00<?, ?it/s]

INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK
Status: processing
INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK
Status: processing
INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK
Status: processing
INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK
Status: processing


...

t:  98%|█████████▊| 3249/3301 [04:10<00:03, 13.30it/s, now=None]

INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK


t:  99%|█████████▊| 3253/3301 [04:11<00:04, 11.65it/s, now=None]

Status: processing


Moviepy - Done !
Moviepy - video ready output/dc0302c1-977c-42cc-8d16-eb6518d12e71.mp4
INFO:     34.142.242.33:0 - "GET /job_status/dc0302c1-977c-42cc-8d16-eb6518d12e71 HTTP/1.1" 200 OK
Status: done
✅ Video ready: http://dc22-34-142-242-33.ngrok-free.app/get_video/dc0302c1-977c-42cc-8d16-eb6518d12e71
